# Graph based Music Recommender. Task 6

For the user with Id 776748 print top-40 recommended tracks. Build music recommendations with the algorithm described in the lesson 3 of the fifth week. Initialize coordinates of vector x_0 corresponding to the user’s vertex and all the vertices from the task 5 with ones and all other coordinates with zeros. Do 5 iterations:

Take alpha = 0.15. and the next balancing functions:

- beta(user, user → artist) = 0.5
- beta(user, user → track) = 0.5
- beta(track, track → track) = 1
- beta(artist, artist → track) = 1

You should receive a table with 3 columns: “name”, “artist” and “rank”. Sort the resulting dataframe in descending order by “rank”, select top 40 recommended tracks, select only the columns “name”, “artist” and “rank”, leave 5 digits after the decimal point in “rank” and print the resulting dataframe.

The part of the result on the sample dataset:

    ...
    Prayer Of The Refugee Artist: Rise Against 1.35278102029
    Eagle Artist: Gotthard 1.21412311013
    21 Guns Artist: Green Day 1.17301653219
    Wait And Bleed Artist: Slipknot 0.921552328559
    Beautiful disaster Artist: 311 0.921552328559
    ...

----

For all subtasks use the same ipython notebook, each subtask should be the continuation of the previous

In [1]:
from pyspark.sql import SparkSession

In [2]:
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

In [3]:
from pyspark.sql import Window
from pyspark.sql.functions import row_number, sum, col, abs, count, desc, asc, when, expr, lit, rank, round as rnd

In [4]:
data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

In [5]:
def norm(df, key1, key2, field, n): 
    
    window = Window.partitionBy(key1).orderBy(col(field).desc())
        
    topsDF = df.withColumn("row_number", row_number().over(window)) \
        .filter(col("row_number") <= n) \
        .drop(col("row_number")) 
        
    tmpDF = topsDF.groupBy(col(key1)).agg(col(key1), sum(col(field)).alias("sum_" + field))
   
    normalizedDF = topsDF.join(tmpDF, key1, "inner") \
        .withColumn("norm_" + field, col(field) / col("sum_" + field)) \
        .cache()

    return normalizedDF

--------------

In [6]:
user = 776748

alpha = 0.15
beta_user_artist = 0.5
beta_user_track = 0.5
beta_track_track = 1
beta_artist_track = 1

---------

# Graph based Music Recommender. Task 1

In [7]:
data1 = data.select(
    col('userId').alias('userId'), 
    col('trackId').alias('trackId1'), 
    col('timestamp').alias('timestamp1')
)

data2 = data.select(
    col('userId').alias('userId'), 
    col('trackId').alias('trackId2'), 
    col('timestamp').alias('timestamp2')
)

weights = (data1.join(data2, "userId")
           .filter(col('trackId1') != col('trackId2'))
           .filter(abs(col('timestamp1') - col('timestamp2')) <= 420).cache()
           .groupBy(col('trackId1'), col('trackId2'))
           .count().alias('count')
          ).cache()

norm_weights = (norm(weights, "trackId1", "trackId2", "count", 1000)
                .withColumn("nxt_value", col("norm_count") * beta_track_track)
               )
track_track = norm_weights.select(
    col("trackId1").alias("source"),
    col("trackId2").alias("target"),
    col("nxt_value"),
).cache()

# Graph based Music Recommender. Task 2

In [8]:
weights = (data
              .groupBy(col("userId"), col("trackId"))
              .count().alias("count")
             ).cache()

norm_weights = (norm(weights, "userId", "trackId", "count", 1000)
                .withColumn("nxt_value", col("norm_count") * beta_user_track)
               )

user_track = norm_weights.select(
    col("userId").alias("source"),
    col("trackId").alias("target"),
    col("nxt_value"),
).cache()

# Graph based Music Recommender. Task 3

In [9]:
weights = (data
           .groupBy(col("userId"), col("artistId"))
           .count().alias("count")
          ).cache()

norm_weights = (norm(weights, "userId", "artistId", "count", 100)
                .withColumn("nxt_value", col("norm_count") * beta_user_artist)
               )

user_artist = norm_weights.select(
    col("userId").alias("source"),
    col("artistId").alias("target"),
    col("nxt_value"),
).cache()

# Graph based Music Recommender. Task 4

In [10]:
weights = (data
           .groupBy(col("artistId"), col("trackId"))
           .count().alias("count")
          ).cache()

norm_weights = (norm(weights, "artistId", "trackId", "count", 100)
                .withColumn("nxt_value", col("norm_count") * beta_artist_track)
               )

artist_track = norm_weights.select(
    col("artistId").alias("source"),
    col("trackId").alias("target"),
    col("nxt_value"),
).cache()

# Graph based Music Recommender. Task 6

In [11]:
edges = (track_track
         .union(user_track)
         .union(user_artist)
         .union(artist_track)
        ).cache()

----------

In [12]:
user_data = data.filter(col("userId") == user)

users = (data
         .select(col("userId").alias("id"))
         .distinct()
         .withColumn("p", when(col("id") == user, 1.0).otherwise(0.0))
        )

tracks = (data
          .select(col("trackId").alias("id"))
          .distinct()
          .join(user_data.select(col("trackId").alias("id"), lit(1).alias("tmp")).distinct(), on="id", how="left")
          .withColumn("p", when(~col("tmp").isNull(), 1.0).otherwise(0.0))
          .select(col("id"), col("p"))
         )

artists = (data
           .select(col("artistId").alias("id"))
           .distinct()
           .join(user_data.select(col("artistId").alias("id"), lit(1).alias("tmp")).distinct(), on="id", how="left")
           .withColumn("p", when(~col("tmp").isNull(), 1.0).otherwise(0.0))
           .select(col("id"), col("p"))
         )

x = (users
     .union(artists)
     .union(tracks)
    ).cache()

---

In [13]:
u = (x
     .withColumn("u_prob", when(col("id") == user, 1.0).otherwise(0.0))
     .select("id", "u_prob") 
    ).cache()


for _ in range(5):
    sigma = (x
             .join(edges, on=expr("id = source"), how="left")
             .na.fill(0.0, ["nxt_value"])
             .withColumn("acc", col("p") * col("nxt_value"))
             .groupBy("target")
             .agg(sum("acc").alias("sigma"))
            )
    x = (u
         .join(sigma, on=expr("id = target"), how="left")
         .na.fill(0.0, ["sigma"])
         .withColumn("nxt_value", alpha*col("u_prob") + (1-alpha) * col("sigma"))
         .select(col("id"), col("nxt_value").alias("p"))
        ).cache()

In [14]:
results = (x
          .where("id != " + str(user))
          .join(meta, on="id")
          .orderBy(col("p").desc())
          .select(col("Name"), col("Artist"), rnd(col("p"), 5).alias("p"))
         ).cache()

In [15]:
for name, artist, p in results.limit(40).collect():
    print("{} {} {}".format(name, artist, p))

Kill The DJ Artist: Green Day 1.42809
Come Out and Play Artist: The Offspring 1.37473
I Hate Everything About You Artist: Three Days Grace 1.37362
Prayer Of The Refugee Artist: Rise Against 1.35278
Eagle Artist: Gotthard 1.21412
21 Guns Artist: Green Day 1.17302
Beautiful disaster Artist: 311 0.92155
Wait And Bleed Artist: Slipknot 0.92155
Here To Stay Artist: Korn 0.91653
Hard Rock Hallelujah Artist: Lordi 0.91653
Nothing Going On Artist: Clawfinger 0.80983
In The End Artist: Linkin Park 0.80292
Numb Artist: Linkin Park 0.80292
Sky is Over Artist: Serj Tankian 0.68799
Kryptonite Artist: 3 Doors Down 0.68799
Take It Out On Me Artist: Thousand Foot Krutch 0.47024
Girls and Boys Artist: Blur 0.40245
Cocaine Artist: Nomy 0.20893
Getting Away With Murder Artist: Papa Roach 0.20648
Artist: Green Day Artist: Green Day 0.01181
Artist: Clawfinger Artist: Clawfinger 0.00472
Artist: The Offspring Artist: The Offspring 0.00472
Artist: Linkin Park Artist: Linkin Park 0.00472
The Vengeful One Artis